In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from google.colab import files
import time


In [ ]:
df = pd.read_csv("cleaned_df.csv")

In [ ]:
df.head(5)

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments,Standardized_gender
0,2014-08-27 11:34:53,46,male,United States,MD,Yes,Yes,No,Sometimes,1-5,...,No,No,Yes,Yes,No,Yes,Yes,Yes,No comments,male
1,2014-08-27 11:35:08,36,male,France,Not Applicable,Yes,Yes,No,No data,6-25,...,No,No,Some of them,Some of them,Maybe,Maybe,Don't know,No,No comments,male
2,2014-08-27 11:35:12,29,male,United States,NY,No,Yes,Yes,Sometimes,100-500,...,Maybe,No,Some of them,Some of them,No,No,No,No,No comments,male
3,2014-08-27 11:35:24,31,male,United States,NC,Yes,No,No,Never,1-5,...,No,No,Some of them,Some of them,No,Maybe,Yes,No,No comments,male
4,2014-08-27 11:35:48,46,male,United States,MA,No,No,Yes,Often,26-100,...,Maybe,No,Some of them,Yes,No,Maybe,No,No,No comments,male


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226 entries, 0 to 1225
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Timestamp                  1226 non-null   object
 1   Age                        1226 non-null   int64 
 2   Gender                     1226 non-null   object
 3   Country                    1226 non-null   object
 4   state                      1226 non-null   object
 5   self_employed              1226 non-null   object
 6   family_history             1226 non-null   object
 7   treatment                  1226 non-null   object
 8   work_interfere             1226 non-null   object
 9   no_employees               1226 non-null   object
 10  remote_work                1226 non-null   object
 11  tech_company               1226 non-null   object
 12  benefits                   1226 non-null   object
 13  care_options               1226 non-null   object
 14  wellness

In [ ]:
selected_features = ['Age', 'Standardized_gender', 'self_employed', 'family_history', 'work_interfere',
                     'no_employees', 'remote_work', 'tech_company', 'benefits',
                     'care_options', 'wellness_program', 'seek_help', 'anonymity',
                     'leave', 'mental_health_consequence', 'phys_health_consequence',
                     'coworkers', 'supervisor', 'mental_health_interview',
                     'phys_health_interview', 'mental_vs_physical', 'obs_consequence']

df_selected = df[selected_features + ['treatment']]

categorical_columns = ['Standardized_gender', 'self_employed', 'family_history', 'work_interfere',
                       'no_employees', 'remote_work', 'tech_company', 'benefits',
                       'care_options', 'wellness_program', 'seek_help', 'anonymity',
                       'leave', 'mental_health_consequence', 'phys_health_consequence',
                       'coworkers', 'supervisor', 'mental_health_interview',
                       'phys_health_interview', 'mental_vs_physical', 'obs_consequence']
df_encoded = pd.get_dummies(df_selected, columns=categorical_columns)


scaler = StandardScaler()

# Стандартизируем возраст
df_encoded['Age'] = scaler.fit_transform(df_encoded[['Age']])



In [ ]:
df_encoded['treatment'] = df_encoded['treatment'].map({'Yes': 1, 'No': 0})

In [ ]:
X = df_encoded.drop('treatment', axis=1)
y= df_encoded['treatment']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)

In [ ]:
# Создаем экземпляр модели логистической регрессии
log_reg = LogisticRegression(max_iter=1000)

# Обучаем модель
log_reg.fit(X_train, y_train)

# Предсказываем на тестовых данных
y_pred_log_reg = log_reg.predict(X_test)

# Оценка модели
print("Логистическая регрессия:")
print("Точность:", accuracy_score(y_test, y_pred_log_reg))
print(classification_report(y_test, y_pred_log_reg))

Логистическая регрессия:
Точность: 0.8179347826086957
              precision    recall  f1-score   support

           0       0.89      0.74      0.81       190
           1       0.76      0.90      0.83       178

    accuracy                           0.82       368
   macro avg       0.83      0.82      0.82       368
weighted avg       0.83      0.82      0.82       368



In [ ]:
# Создаем экземпляр модели Random Forest
rf_model = RandomForestClassifier(random_state=99)

# Обучаем модель
rf_model.fit(X_train, y_train)

# Предсказываем на тестовых данных
y_pred_rf = rf_model.predict(X_test)

# Оценка модели
print("Random Forest:")
print("Точность:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Random Forest:
Точность: 0.8179347826086957
              precision    recall  f1-score   support

           0       0.90      0.73      0.80       190
           1       0.76      0.92      0.83       178

    accuracy                           0.82       368
   macro avg       0.83      0.82      0.82       368
weighted avg       0.83      0.82      0.82       368



In [ ]:
rf_model = RandomForestClassifier(random_state=99)

# Задаем гиперпараметры для поиска
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt', 'log2'],
}



grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid,
                               cv=5, scoring='accuracy', verbose=2, n_jobs=-1)


grid_search_rf.fit(X_train, y_train)


print("Лучшие параметры Random Forest:", grid_search_rf.best_params_)
print("Лучшая точность:", grid_search_rf.best_score_)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Лучшие параметры Random Forest: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 100}
Лучшая точность: 0.8239358085135319


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [ ]:
import xgboost as xgb

# Создаем экземпляр модели XGBoost
xgb_model = xgb.XGBClassifier(random_state=99)


xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost:")
print("Точность:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))


XGBoost:
Точность: 0.782608695652174
              precision    recall  f1-score   support

           0       0.83      0.73      0.78       190
           1       0.74      0.84      0.79       178

    accuracy                           0.78       368
   macro avg       0.79      0.78      0.78       368
weighted avg       0.79      0.78      0.78       368



In [ ]:
xgb_model = xgb.XGBClassifier(random_state=99)

# Задаем гиперпараметры для поиска
param_dist_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0]
}

# Инициализируем RandomizedSearchCV
random_search_xgb = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist_xgb,
                                        n_iter=50, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

random_search_xgb.fit(X_train, y_train)

print("Лучшие параметры XGBoost:", random_search_xgb.best_params_)
print("Лучшая точность:", random_search_xgb.best_score_)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Лучшие параметры XGBoost: {'subsample': 0.7, 'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.01, 'colsample_bytree': 1.0}
Лучшая точность: 0.8286141710866313


In [ ]:
# Создаем экземпляр модели SVM
svm_model = SVC(random_state=99)

svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)

print("SVM:")
print("Точность:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


SVM:
Точность: 0.8233695652173914
              precision    recall  f1-score   support

           0       0.93      0.71      0.81       190
           1       0.75      0.94      0.84       178

    accuracy                           0.82       368
   macro avg       0.84      0.83      0.82       368
weighted avg       0.85      0.82      0.82       368



In [ ]:
svm_model = SVC(random_state=99)

# Задаем гиперпараметры для поиска
param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],  # разные ядра
    'gamma': ['scale', 'auto']
}

# Инициализируем GridSearchCV
grid_search_svm = GridSearchCV(estimator=svm_model, param_grid=param_grid_svm,
                                cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

grid_search_svm.fit(X_train, y_train)

print("Лучшие параметры SVM:", grid_search_svm.best_params_)
print("Лучшая точность:", grid_search_svm.best_score_)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Лучшие параметры SVM: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Лучшая точность: 0.8332789337685298


In [ ]:
pip install lightgbm


In [ ]:
pip install dask[dataframe]


In [ ]:
X_train.columns = X_train.columns.str.replace('[^a-zA-Z0-9]', '_', regex=True)


In [ ]:

# Настройка параметров для Gradient Boosting
gb_params = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}

# Оптимизация гиперпараметров для Gradient Boosting
gb_clf = GradientBoostingClassifier()
gb_grid = GridSearchCV(gb_clf, gb_params, cv=5, scoring='accuracy', n_jobs=-1)
gb_grid.fit(X_train, y_train)

# Лучшие параметры и точность
print("Лучшие параметры Gradient Boosting:", gb_grid.best_params_)
print("Лучшая точность Gradient Boosting:", gb_grid.best_score_)




Лучшие параметры Gradient Boosting: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
Лучшая точность Gradient Boosting: 0.8286141710866313


In [ ]:
# Настройка параметров для LightGBM
lgb_params = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [-1, 5, 10],
    'num_leaves': [31, 63]
}

# Оптимизация гиперпараметров для LightGBM
lgb_clf = LGBMClassifier()
lgb_grid = GridSearchCV(lgb_clf, lgb_params, cv=5, scoring='accuracy', n_jobs=-1)
lgb_grid.fit(X_train, y_train)

# Лучшие параметры и точность
print("Лучшие параметры LightGBM:", lgb_grid.best_params_)
print("Лучшая точность LightGBM:", lgb_grid.best_score_)

[LightGBM] [Info] Number of positive: 442, number of negative: 416
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000414 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 166
[LightGBM] [Info] Number of data points in the train set: 858, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.515152 -> initscore=0.060625
[LightGBM] [Info] Start training from score 0.060625
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [ ]:
df_encoded = pd.read_csv("df_significant.csv")

In [ ]:
df_encoded.head(5)

,work_interfere_No data,work_interfere_Sometimes,work_interfere_Never,family_history_Yes,family_history_No,work_interfere_Often,care_options_Yes,benefits_Yes,Standardized_gender_female,benefits_Don't know,...,mental_health_consequence_Yes,seek_help_Yes,mental_vs_physical_Don't know,wellness_program_Yes,leave_Don't know,Standardized_gender_other,no_employees_6-25,coworkers_Yes,obs_consequence_No,treatment
0,False,True,False,True,False,False,False,True,False,False,...,False,False,False,True,False,False,False,True,False,0
1,True,False,False,True,False,False,False,False,False,False,...,False,False,True,True,False,False,True,False,True,0
2,False,True,False,True,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,True,1
3,False,False,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,0
4,False,False,False,False,True,True,True,True,False,False,...,False,False,False,False,True,False,False,False,True,1


In [ ]:
X = df_encoded.drop('treatment', axis=1)
y= df_encoded['treatment']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)

In [ ]:
import numpy as np

# Проверка распределения классов
unique_classes, class_counts = np.unique(y_train, return_counts=True)
class_distribution = dict(zip(unique_classes, class_counts))

print("Распределение классов в y_train:", class_distribution)


Распределение классов в y_train: {0: 416, 1: 442}


In [ ]:
df_encoded['treatment'].head(10)


,treatment
0,0
1,0
2,1
3,0
4,1
5,1
6,1
7,1
8,0
9,1


In [ ]:
df_encoded['treatment'] = df_encoded['treatment'].map({'Yes': 1, 'No': 0})


In [ ]:
# Проверка распределения классов в y_train
unique_classes = np.unique(y_train)
print("Уникальные классы в y_train:", unique_classes)


Уникальные классы в y_train: [0 1]


In [ ]:
# Проверка уникальных значений в колонке treatment
print("Уникальные значения в колонке treatment:", df_encoded['treatment'].unique())


Уникальные значения в колонке treatment: [nan]


In [ ]:
# Создаем экземпляр модели логистической регрессии
log_reg = LogisticRegression(max_iter=1000)

# Обучаем модель
log_reg.fit(X_train, y_train)

# Предсказываем на тестовых данных
y_pred_log_reg = log_reg.predict(X_test)

# Оценка модели
print("Логистическая регрессия:")
print("Точность:", accuracy_score(y_test, y_pred_log_reg))
print(classification_report(y_test, y_pred_log_reg))

Логистическая регрессия:
Точность: 0.8396739130434783
              precision    recall  f1-score   support

           0       0.93      0.75      0.83       190
           1       0.78      0.94      0.85       178

    accuracy                           0.84       368
   macro avg       0.85      0.84      0.84       368
weighted avg       0.85      0.84      0.84       368



In [ ]:
# Создаем экземпляр модели Random Forest
rf_model = RandomForestClassifier(random_state=99)

# Обучаем модель
rf_model.fit(X_train, y_train)

# Предсказываем на тестовых данных
y_pred_rf = rf_model.predict(X_test)

# Оценка модели
print("Random Forest:")
print("Точность:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Random Forest:
Точность: 0.8125
              precision    recall  f1-score   support

           0       0.87      0.75      0.80       190
           1       0.77      0.88      0.82       178

    accuracy                           0.81       368
   macro avg       0.82      0.81      0.81       368
weighted avg       0.82      0.81      0.81       368



In [ ]:
rf_model = RandomForestClassifier(random_state=99)

# Задаем гиперпараметры для поиска
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt', 'log2'],
}



grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid,
                               cv=5, scoring='accuracy', verbose=2, n_jobs=-1)


grid_search_rf.fit(X_train, y_train)


print("Лучшие параметры Random Forest:", grid_search_rf.best_params_)
print("Лучшая точность:", grid_search_rf.best_score_)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Лучшие параметры Random Forest: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 100}
Лучшая точность: 0.8274037807697538


In [ ]:
import xgboost as xgb

# Создаем экземпляр модели XGBoost
xgb_model = xgb.XGBClassifier(random_state=99)


xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost:")
print("Точность:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))


XGBoost:
Точность: 0.7853260869565217
              precision    recall  f1-score   support

           0       0.83      0.74      0.78       190
           1       0.75      0.84      0.79       178

    accuracy                           0.79       368
   macro avg       0.79      0.79      0.79       368
weighted avg       0.79      0.79      0.79       368



In [ ]:
xgb_model = xgb.XGBClassifier(random_state=99)

# Задаем гиперпараметры для поиска
param_dist_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0]
}

# Инициализируем RandomizedSearchCV
random_search_xgb = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist_xgb,
                                        n_iter=50, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

random_search_xgb.fit(X_train, y_train)

print("Лучшие параметры XGBoost:", random_search_xgb.best_params_)
print("Лучшая точность:", random_search_xgb.best_score_)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Лучшие параметры XGBoost: {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.01, 'colsample_bytree': 1.0}
Лучшая точность: 0.8286141710866313


In [ ]:
# Создаем экземпляр модели SVM
svm_model = SVC(random_state=99)

svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)

print("SVM:")
print("Точность:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


SVM:
Точность: 0.8342391304347826
              precision    recall  f1-score   support

           0       0.94      0.73      0.82       190
           1       0.76      0.95      0.85       178

    accuracy                           0.83       368
   macro avg       0.85      0.84      0.83       368
weighted avg       0.85      0.83      0.83       368



In [ ]:
svm_model = SVC(random_state=99)

# Задаем гиперпараметры для поиска
param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],  # разные ядра
    'gamma': ['scale', 'auto']
}

# Инициализируем GridSearchCV
grid_search_svm = GridSearchCV(estimator=svm_model, param_grid=param_grid_svm,
                                cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

grid_search_svm.fit(X_train, y_train)

print("Лучшие параметры SVM:", grid_search_svm.best_params_)
print("Лучшая точность:", grid_search_svm.best_score_)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Лучшие параметры SVM: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Лучшая точность: 0.8286141710866313


In [ ]:
pip install lightgbm


In [ ]:
pip install dask[dataframe]


In [ ]:
X_train.columns = X_train.columns.str.replace('[^a-zA-Z0-9]', '_', regex=True)


In [ ]:

# Настройка параметров для Gradient Boosting
gb_params = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}

# Оптимизация гиперпараметров для Gradient Boosting
gb_clf = GradientBoostingClassifier()
gb_grid = GridSearchCV(gb_clf, gb_params, cv=5, scoring='accuracy', n_jobs=-1)
gb_grid.fit(X_train, y_train)

# Лучшие параметры и точность
print("Лучшие параметры Gradient Boosting:", gb_grid.best_params_)
print("Лучшая точность Gradient Boosting:", gb_grid.best_score_)




Лучшие параметры Gradient Boosting: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
Лучшая точность Gradient Boosting: 0.8286141710866313


In [ ]:
# Настройка параметров для LightGBM
lgb_params = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [-1, 5, 10],
    'num_leaves': [31, 63]
}

# Оптимизация гиперпараметров для LightGBM
lgb_clf = LGBMClassifier()
lgb_grid = GridSearchCV(lgb_clf, lgb_params, cv=5, scoring='accuracy', n_jobs=-1)
lgb_grid.fit(X_train, y_train)

# Лучшие параметры и точность
print("Лучшие параметры LightGBM:", lgb_grid.best_params_)
print("Лучшая точность LightGBM:", lgb_grid.best_score_)

[LightGBM] [Info] Number of positive: 442, number of negative: 416
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000282 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 60
[LightGBM] [Info] Number of data points in the train set: 858, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.515152 -> initscore=0.060625
[LightGBM] [Info] Start training from score 0.060625
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [ ]:
model_results = {
    "Model": ["Logistic Regression", "Random Forest", "XGBoost", "SVM", "Gradient Boosting", "LightGBM"],
    "Accuracy": [
        accuracy_score(y_test, y_pred_log_reg),  # Логистическая регрессия
        accuracy_score(y_test, y_pred_rf),       # Random Forest
        accuracy_score(y_test, y_pred_xgb),      # XGBoost
        accuracy_score(y_test, y_pred_svm),      # SVM
        gb_grid.best_score_,                     # Gradient Boosting
        lgb_grid.best_score_                     # LightGBM
    ],
    "Best Params": [
        None,                        # Логистическая регрессия (нет гиперпараметров)
        grid_search_rf.best_params_,  # Random Forest
        random_search_xgb.best_params_, # XGBoost
        grid_search_svm.best_params_,   # SVM
        gb_grid.best_params_,           # Gradient Boosting
        lgb_grid.best_params_           # LightGBM
    ]
}

# Преобразуем результаты в таблицу pandas для удобного отображения
df_results = pd.DataFrame(model_results)

# Вывод таблицы с результатами
print("Результаты всех моделей:")
print(df_results)

Результаты всех моделей:
                 Model  Accuracy  \
0  Logistic Regression  0.839674   
1        Random Forest  0.812500   
2              XGBoost  0.785326   
3                  SVM  0.834239   
4    Gradient Boosting  0.828614   
5             LightGBM  0.827445   

                                         Best Params  
0                                               None  
1  {'max_depth': 5, 'max_features': 'sqrt', 'min_...  
2  {'subsample': 1.0, 'n_estimators': 200, 'max_d...  
3   {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}  
4  {'learning_rate': 0.01, 'max_depth': 3, 'n_est...  
5  {'learning_rate': 0.01, 'max_depth': 5, 'n_est...  


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Функция для получения метрик
def get_metrics(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"{model_name}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}\n")

    return accuracy, precision, recall, f1


In [ ]:

# Перед разделением на train/test заменим имена колонок правильно
X.columns = X.columns.str.replace('[^a-zA-Z0-9]', '_', regex=True)

# Теперь разделим на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)

# Функция для оценки модели
def evaluate_model(model, X_train, X_test, y_train, y_test):
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time

    start_time = time.time()
    y_pred = model.predict(X_test)
    predict_time = time.time() - start_time

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Training Time": train_time,
        "Prediction Time": predict_time
    }

# Проверяем работу для моделей
log_reg_results = evaluate_model(log_reg, X_train, X_test, y_train, y_test)
rf_results = evaluate_model(rf_model, X_train, X_test, y_train, y_test)
xgb_results = evaluate_model(xgb_model, X_train, X_test, y_train, y_test)

# Выводим результаты
print("Логистическая регрессия:", log_reg_results)
print("Random Forest:", rf_results)
print("XGBoost:", xgb_results)


Логистическая регрессия: {'Accuracy': 0.8396739130434783, 'Precision': 0.7767441860465116, 'Recall': 0.9382022471910112, 'F1 Score': 0.8498727735368957, 'Training Time': 0.022746562957763672, 'Prediction Time': 0.0042378902435302734}
Random Forest: {'Accuracy': 0.8125, 'Precision': 0.7658536585365854, 'Recall': 0.8820224719101124, 'F1 Score': 0.8198433420365535, 'Training Time': 0.9324972629547119, 'Prediction Time': 0.03004288673400879}
XGBoost: {'Accuracy': 0.7853260869565217, 'Precision': 0.7487437185929648, 'Recall': 0.8370786516853933, 'F1 Score': 0.7904509283819628, 'Training Time': 0.7046070098876953, 'Prediction Time': 0.05494523048400879}
